In [357]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import numpy as np
import numpy.matlib
import random
import os
import pathlib
import time

## Classes

In [358]:
class Individual:
  """
  A class that represents an individual solution.

  Attributes:
    chromosome (list): The chromosome of the individual.
    weights (list): The weights of the individual.
    fitness (float): The fitness of the individual.

  -----------
    c (numpy.ndarray): 1D list with values in [0,1] interval.
    w (numpy.ndarray): 2D list with values in [-1,1] interval.
    fitness (float): The fitness of the individual.
  """
  def __init__(self, chromosome, weights, fitness=math.inf):
    self._chromosome = chromosome
    self._weights = weights
    if fitness < 0.0:
      raise ValueError('The fitness of an individual cannot be negative.')
    else:
      self._fitness = fitness
  
  def __str__(self):
    return f"""
    Individual:
        Feature-selection: {self._chromosome}
        Weights: {self._weights}
        Fitness: {self._fitness}
    """
  @property
  def chromosome(self):
    return self._chromosome
  
  @property
  def weights(self):
    return self._weights

  @property
  def fitness(self):
    return self._fitness

  @fitness.setter
  def fitness(self,f):
    self._fitness = f

In [359]:
class Particle(Individual):
  """
  A class that represents an individual solution in Swarm-based problems.

  Attributes:
  -----------
    (same as Individual class), and additional attributes:

    best_fitness (float): The best fitness of the individual.
    best_weights_position (numpy.ndarray): The best weight position of the individual.
    best_chromosome_position (numpy.ndarray): The best chromosome position of the individual.
    weights_velocity (numpy.ndarray): The velocity of the weights of the individual.
    chromosome_velocity (numpy.ndarray): The velocity of the chromosome of the individual.  
  """
  
  def __init__(self, chromosome, weights, fitness=math.inf):
    Individual.__init__(self, chromosome, weights, fitness)
    self._best_fitness = fitness # At the beginning, the best fitness is the input fitness
    self._best_weights_position = weights # By default, the best weight position is the starting weight position.
    self._best_chromosome_position = chromosome # By default, the best chromosome position is the starting chromosome position.
    self._weights_velocity = np.zeros(weights.shape) # All swarm's weights velocities are set to 0 at the start
    self._chromosome_velocity = np.zeros(chromosome.shape) # All swarm's chromosome velocities are set to 0 at the start

  def __str__(self):
    return f"""
    Particle:
        Feature-selection: {self.chromosome}
        Weights: {self.weights}
        Fitness: {self.fitness}
        Actual weights velocity: {self._weights_velocity}
        Actual feature-selection velocity: {self._chromosome_velocity}
        Best fitness: {self._best_fitness}
        Best weights: {self._best_weights_position}
        Best feature-selection: {self._best_chromosome_position}
    """

  @property
  def best_fitness(self):
    return self._best_fitness

  @property
  def best_weights_position(self):
    return self._best_weights_position
  
  @property
  def best_chromosome_position(self):
    return self._best_chromosome_position
  
  @property
  def weights_velocity(self):
    return self._weights_velocity

  @property
  def chromosome_velocity(self):
    return self._chromosome_velocity

  @best_fitness.setter
  def best_fitness(self,f):
    self._best_fitness = f


  @best_weights_position.setter
  def best_weights_position(self,w):
    self._best_weights_position = w

  @best_chromosome_position.setter
  def best_chromosome_position(self,c):
    self._best_chromosome_position = c

In [360]:
class Population:
  """
  A class that represents a population of individual solutions.

  Attributes:
  -----------
    size (int): The size of the population.
    gene_list (list): The list of individuals solutions.
    fitness_list (list): The list of fitness of the individuals.
    best_gene (Individual): The best individual of the population.
  
  Methods:
  -----------
    insert_best_gene(Individual): Inserts the best individual of the population into the gene_list and fitness_list.
    add_gene_to_list(Individual): Adds an individual to the gene_list.
    add_fitness_to_list(float): Adds a fitness to the fitness_list.
  """

  def __init__(self, size:int, K:int = 0, D:int = 0, is_empty:bool = False, is_swarm:bool = False):
    self._size = size
    self._best_gene = None

    if is_empty == True:
      self._genes_list = []
      self._fitness_list = []
    else:
      self._fitness_list = [math.inf]*size
      if is_swarm == False:
        self._genes_list = [create_individual(K,D) for _ in range(size)]
      else:
        self._genes_list = [create_particle(K,D) for _ in range(size)]
  
  def __str__(self):
    return f"""
    Population:
        Size: {self._size}
        Gene list: {self._genes_list}
        Fitness list: {self._fitness_list}
        Best gene: {self._best_gene}
    """
  @property
  def size(self):
    return self._size
  
  @property
  def genes_list(self):
    return self._genes_list

  @property
  def fitness_list(self):
    return self._fitness_list

  @property
  def best_gene(self):
    return self._best_gene

  @best_gene.setter
  def best_gene(self,gen):  
    self._best_gene = gen

  # Methods
  def add_gene_to_list(self,gen):
    self.genes_list.append(gen)

  def add_fitness_to_list(self,fitness):
    self.fitness_list.append(fitness)

  def insert_best_gene(self,gen):
    self.best_gene = gen
    self.genes_list.append(gen)
    self.fitness_list.append(gen.fitness)

In [361]:
class Swarm(Population):
  """
  A class that represents a population of solutions in Swarm-based problems.

  Attributes:
  -----------
    (same as Population), and additional attributes:

    global_best_fitness (float): The best fitness of the population.
    global_best_weights (numpy.ndarray): The best weight position of the population.
    global_best_chromosome (numpy.ndarray): The best chromosome position of the population.    
  
  Methods:
  -----------

  """

  def __init__(self, size:int, K:int = 0, D:int = 0, is_empty:bool = False):
    Population.__init__(self, size, K, D, is_empty, is_swarm=True)
    self._global_best_fitness = math.inf
    self._global_best_weights = None
    self._global_best_chromosome = None

  def __str__(self):
    return f"""
    Swarm:
        Size: {self._size}
        Particles list: {self.genes_list}
        Fitness list: {self.fitness_list}
        Best particle: {self.best_gene}
        Global best fitness: {self._global_best_fitness}
        Global best weights: {self._global_best_weights}
        Global best feature-selection: {self._global_best_chromosome}
    """
  @property
  def global_best_fitness(self):
    return self._global_best_fitness

  @property
  def global_best_weights(self):
    return self._global_best_weights

  @property
  def global_best_chromosome(self):
    return self._global_best_chromosome

  @global_best_fitness.setter
  def global_best_fitness(self,f):
    self._global_best_fitness = f
  
  @global_best_weights.setter
  def global_best_weights(self,w):
    self._global_best_weights = w
  
  @global_best_chromosome.setter
  def global_best_chromosome(self,c):
    self._global_best_chromosome = c

In [362]:
class Reef:
  """
  A class that represents a population of solutions in Evolutionary-based problems.

  Attributes:
  ----------- 
    size (int): The size of the population.
    genes_list (list): The list of individuals solutions.
    fitness_list (list): The list of fitness of the individuals.
    best_gene (Individual): The best individual of the population.
    
  Methods:
  -----------

  """

  def __init__(self, size:int, rate:float, K:int, D:int):
    self._size = size
    self._free_occupied_rate = rate
    self._fitness_list = [math.inf]*size
    self._best_gene = None
    self._genes_list = np.full([size], None) # Empty reef

    occupiedHoles = int(size * rate) # Partially occupy the reef
    self._genes_list[:occupiedHoles] = create_individual(K,D)

  def __str__(self):
    return f"""
    Reef:
        Size: {self._size}
        Free-occupied rate: {self._free_occupied_rate}
        Corals list: {self._genes_list}
        Fitness list: {self._fitness_list}
        Best coral: {self._best_gene}
    """
  
  @property
  def size(self):
    return self._size

  @property
  def free_occupied_rate(self):
    return self._free_occupied_rate
  
  @property
  def genes_list(self):
    return self._genes_list

  @property
  def fitness_list(self):
    return self._fitness_list

  @property
  def best_gene(self):
    return self._best_gene
  
  @best_gene.setter
  def best_gene(self,gen):
    self._best_gene = gen
  
  # Methods
  def add_gene_to_list(self,gen):
    self.genes_list.append(gen)

  def add_fitness_to_list(self,fitness):
    self.fitness_list.append(fitness)

  def insert_best_gene(self,gen):
    self.best_gene = gen
    self.genes_list.append(gen)
    self.fitness_list.append(gen.fitness)

## Functions

### NumPy custom functions

In [363]:
def sumMatrixes(mat1,mat2):
  return np.add(mat1,np.asarray(mat2))

def subtractMatrixes(mat1,mat2):
  return np.subtract(mat1,np.asarray(mat2))

def dotMultiply(mat1,mat2):
  return np.multiply(mat1,np.asarray(mat2))

def multiplyMatrixes(mat1,mat2):
  return np.matmul(mat1,np.asarray(mat2))

def dotDivide(mat1,mat2):
  return np.divide(mat1,np.asarray(mat2))

### Population

#### Initialisation

In [364]:
def random_C_generator(K):
  # randint returns from: inclusive -[low,high)- exclusive.
  return np.random.randint(low= 0, high= 1 + 1, size=[K])

def random_W_generator(K,D):
  return np.random.uniform(low= -1, high= 1, size=[K,D])

In [365]:
def create_individual(K:int = 0, D:int = 0):
  return Individual(random_C_generator(K), random_W_generator(K,D))

def create_particle(K:int = 0, D:int = 0):
  return Particle(random_C_generator(K), random_W_generator(K,D))

def create_population(size:int, K:int = 0, D:int = 0, is_empty:bool = False):
  return Population(size,K,D, is_empty)

def create_swarm(size:int, K:int = 0, D:int = 0, is_empty:bool = False):
  return Swarm(size,K,D, is_empty)

def create_reef(size:int, rate:float, K:int = 0, D:int = 0):
  return Reef(size, rate, K, D)

#### Reproduction

In [366]:
def InternalReproduction(gen: Individual):
  # Mutate chromosome 
  mutate_point = np.random.randint(0, len(gen.chromosome)) # Choose a random point to mutate
  gen.chromosome[mutate_point] = 1 - gen.chromosome[mutate_point] # Mutate the point
  
  # Mutate weights
  nRows = gen.weights.shape[0] 
  nColumns = gen.weights.shape[1] 
  mutate_column = np.random.randint(0, nColumns) # Choose a random column to mutate
  
  gen.weights[:,mutate_column] = np.random.uniform(-1,1,nRows) # Mutate the column

In [367]:
def ExternalReproduction(father:Individual, mother:Individual):
  crossover_point = np.random.randint(0, len(father.chromosome))
  crossover_column = random.randint(0, len(father.weights[0]))
  
  # Offsprings are created by combining father and mother chromosomes and weights
  offspring1 = Individual( chromosome = np.concatenate((father.chromosome[:crossover_point], mother.chromosome[crossover_point:])), 
                           weights = np.concatenate((father.weights[:,:crossover_column], mother.weights[:,crossover_column:]), axis=1) ) 
  
  offspring2 = Individual( chromosome = np.concatenate((mother.chromosome[:crossover_point], father.chromosome[crossover_point:])), 
                           weights = np.concatenate((mother.weights[:,:crossover_column], father.weights[:,crossover_column:]), axis=1) ) 
                       
  return [offspring1,offspring2] 

In [368]:
def RouletteWheelSelection(population:Population): 
  # Create an empty population for the chosen individuals of the Roulette Wheel
  roulettePopulation = create_population(population.size, is_empty=True)
  
  # Make sure we don't lose our best gene in the roulette 
  roulettePopulation.insert_best_gene(population.best_gene)

  print(f"Roulette population after inserting best gene: {roulettePopulation}")

  # Total population fitness (S)
  S = sum([individual.fitness for individual in population.genes_list])
  
  # Population chromosomes' relative probabilities
  rel_prob = [individual.fitness/S for individual in population.genes_list]

  for _ in range(population.size - 1): # -1 because we already inserted the best gene
    # Generate a random uniform number (r) - (0,1]
    r = np.random.uniform() 
    # Find the first index for which q_i < r
    for index,individual in enumerate(population.genes_list): 
      r -= rel_prob[index] 
      if r < 0:
        roulettePopulation.add_gene_to_list(individual) 
        roulettePopulation.add_fitness_to_list(individual.fitness) 
        break

  print(f"Roulette Wheel final population:")
  for index,i in enumerate(roulettePopulation.genes_list):
    print(f"{index}: {i.chromosome} - fitness: {i.fitness}")
  return roulettePopulation  

In [369]:
''' This functions reproduces a population.
    It crossover the parents (external reproduction) and mutate the offspring (internal reproduction).
    Returns the input population, updated with the crossover and mutated childs '''

def ReproducePopulation(population: Population):
  childs = [] # list to store future offsprings
  crossover_probability = np.random.uniform() # crossover probability
  mutation_probability = np.random.uniform() # mutation probability
  gene_list_size = len(population.genes_list)
  print("... Reproducing population ...")
  for index,i in enumerate(population.genes_list):
    print(f"{index} - {i.chromosome}")

  for index,individual in enumerate(population.genes_list):
    # Generate a random mother if the crossover probability is met
    if(np.random.random() >= crossover_probability):
      random_mother_index = np.random.randint(0,gene_list_size) # Choose a random mother
      contador = 0

      while(random_mother_index == index): # If the mother is the same as the father, choose another one
        print(f"Contador - {contador}, old random_mother_index - {random_mother_index}, index - {index}")
        random_mother_index = np.random.randint(0,gene_list_size)
        contador += 1
        print(f"Contador - {contador}, new random_mother_index - {random_mother_index}, index - {index}")
      #end while
      
      # Crossover parents
      print("Creating two childs!")
      mother = population.genes_list[random_mother_index]
      offsprings = ExternalReproduction(individual, mother)

      # Mutate offsprings if probability is over the mutation probability
      rand_value = np.random.random()
      if(rand_value >= mutation_probability): 
        InternalReproduction(offsprings[0])

      if(rand_value >= mutation_probability):
        InternalReproduction(offsprings[1])

      childs.append(offsprings[0])
      childs.append(offsprings[1])
  
  # Add the childs to the population genes list
  population.genes_list.extend(childs)

  print(f"Reproduced final population:")
  for index,i in enumerate(population.genes_list):
    print(f"{index} - {i.chromosome} - fitness: {i.fitness}")
  return population

#### Swarm

In [370]:
def UpdateVelocitiesAndPositionsPSO(swarm: Swarm, w:float, c1:float, c2:float):
  for index, particle in enumerate(swarm.genes_list): # For each particle in the swarm

    # Velocities
    r1 = np.random.uniform()
    r2 = np.random.uniform()

    # Chromosome loop
    for k in range(len(particle.chromosome)):
      # Update chromosome velocity 
      particle.chromosome_velocity[k] = ( 
                                          (w * particle.chromosome_velocity[k]) +
                                          (c1*r1*(particle.best_chromosome_position[k] - particle.chromosome[k])) +
                                          (c2*r2*(swarm.global_best_chromosome[k] - particle.chromosome[k]))
                                        )
      
      #print(f"Particle {index}  actual Velocity {k}: {p.actual_c_velocity[k]}")

      velocityProbability = 2 / math.pi * math.atan((math.pi*0.5)*particle.chromosome_velocity[k]) #|2⁄𝜋 × arctan ((𝜋 2) × 𝑉𝑡+1
      #print(f"Particle {index} velocityProbability {k}: {velocityProbability}")

      # Update to next position
      if(np.random.uniform() < velocityProbability):
        particle.chromosome[k] = 1
      else:
        particle.chromosome[k] = 0
    # end chromosome loop

    # Weight loop
    for i in range((particle.weights).shape[0]):
      for j in range((particle.weights).shape[1]):
        particle.weights_velocity[i,j] = ( 
                                          (w * particle.weights_velocity[i,j]) +
                                          (c1*r1*(particle.best_weights_position[i,j] - particle.weights[i,j])) +
                                          (c2*r2*(swarm.global_best_weights[i,j] - particle.weights[i,j]))
                                        )

        # Update to next position
        particle.weights[i,j] += particle.weights_velocity[i,j]
    # end weight loop
  # end particle loop
  return swarm

## Data Processing

In [371]:
CSV_NAME = "parkinsons.csv"

PORTATIL_DEV = False

if PORTATIL_DEV:
    CSV_PATH = "C:/Users/david/OneDrive/Escritorio/MrRobot/IITV/4/TFG/data-Vito-PC/"
else:
    CSV_PATH = "E:/Perfil/OneDrive/Escritorio/MrRobot/IITV/4/TFG/data-Vito-PC/"
    
df = pd.read_csv(CSV_PATH + CSV_NAME, sep=" ", header=None)

end = df.shape[1]
X = df.iloc[:, :end-1] # iloc[rows,[cols_start,cols_end)] <- Dataframe object
Y = df.iloc[:, end-1] # iloc[rows,col_index] <- Dataframe series

# IMPORTANT, otherwise we would be using DataFrames and matrix operations won't work
X = X.to_numpy()
Y = Y.to_numpy()
J = len(np.unique(Y))

N,K = X.shape[0],X.shape[1] 

In [372]:
# Scaling X (min-max normalization)
X_scaled = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

# First partition training/test
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42) # 20% test data

# Second partition validation
X_trainVal, X_testVal, Y_trainVal, Y_testVal = train_test_split(X_train, Y_train, test_size=0.2, random_state=42) # 20% validation data

## ELM

In [373]:
# Computes the fitness for a given Individual using Extreme Learning Machine (ELM) model
#FIXME: Use feature selection weights to reduce the dimensionality of the input data
def ComputeIndividualFitness(individual:Individual, D: int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
    # W
    individualWeights = individual.weights
    #print(f"Individual weights: {individualWeights} - Shape: {individualWeights.shape}")

    # Feature-selection Bias given Gene's chromosome
    Bias = np.random.uniform(low= 0, high= 1, size=[D,1])

    # Amplify the matrix to the size of Xtraining and Xtest
    BiasTrainingMatrix = np.matlib.repmat(Bias,1,trainingX.shape[0])
    BiasTestMatrix = np.matlib.repmat(Bias,1,testX.shape[0])

    # Transpose BiasMatrix
    BiasTrainingMatrix = np.transpose(BiasTrainingMatrix)
    BiasTestMatrix = np.transpose(BiasTestMatrix)

    # H (Sigmoide function) 
    activationTraining = multiplyMatrixes(trainingX, individualWeights) + BiasTrainingMatrix
    activationTest = multiplyMatrixes(testX, individualWeights) + BiasTestMatrix
    
    H_Training = dotDivide( 1, (1 + np.exp(-activationTraining)) )    
    H_Test = dotDivide( 1, (1 + np.exp(-activationTest)) )

    # Beta
    aux = multiplyMatrixes( np.transpose(H_Training) , H_Training )
    delta = np.identity(aux.shape[0]) * 10e-3

    inverse = np.linalg.inv(dotDivide(np.identity(D) , C) + delta + aux)

    # Complete Formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    Beta = multiplyMatrixes(inverse, np.transpose(H_Training))
    Beta = multiplyMatrixes(Beta, trainingY)

    # Output
    Y_predicted = multiplyMatrixes(H_Test, Beta)
    fitness = np.linalg.norm(Y_predicted - testY)

    return fitness

In [374]:
# Computes and udpates the fitnesses of a given Population 
def ComputePopulationFitness(population: Population, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  # Clear previous fitnesses
  population.fitness_list.clear()

  print("\n...Computing fitnesses...")
  for index,individual in enumerate(population.genes_list):
    fitness_i = ComputeIndividualFitness(individual, D, C, trainingX, trainingY, testX, testY)
    print(f"Individual {index} old fitness: {fitness_i}")
    individual.fitness = fitness_i # Update fitness
    print(f"Individual {index} new fitness: {fitness_i}")
    population.add_fitness_to_list(fitness_i)

    # Update Best Gene in Population
    if population.best_gene is None:
      print(f"New best gene after None: {individual.chromosome}  - {individual.fitness}")
      population.best_gene = individual
    else:
      if individual.fitness < population.best_gene.fitness:
        print(f"New best gene found: {individual.chromosome}  - {individual.fitness}")
        population.best_gene = individual
  # end for

In [375]:
# Computes and udpates the fitnesses of a given Swarm
def ComputeSwarmFitness(swarm: Swarm, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  # Clear previous fitnesses
  swarm.fitness_list.clear()

  for index,particle in enumerate(swarm.genes_list):    
    fitness_i = ComputeIndividualFitness(particle, D, C, trainingX, trainingY, testX, testY)
    particle.setFitness(fitness_i)
    swarm.addFitnessToList(fitness_i)

    # Update personal bests and global best
    if particle.fitness < particle.best_fitness:
      particle.best_fitness = particle.fitness
      
      if particle.fitness < swarm.best_gene.fitness:
        swarm.best_gene = particle

  # Set Swarm's best chromosome and weights
  swarm.global_best_chromosome = swarm.best_gene.chromosome
  swarm.global_best_weights = swarm.best_gene.weights
  print("Global best fitness: ", swarm.best_gene.fitness)

In [376]:
#FIXME: Use feature selection weights to reduce the dimensionality of the input data
def TrainModelAndOutputResults(individual:Individual, D: int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
    print("Starting training...")
    # W
    weights = individual.weights 

    # Bias
    Bias = np.random.uniform(low= 0, high= 1, size=[D,1])

    # Amplify the matrix to the size of Xtraining and Xtest
    BiasTrainingMatrix = np.matlib.repmat(Bias,1,trainingX.shape[0])
    BiasTestMatrix = np.matlib.repmat(Bias,1,testX.shape[0])

    # Transpose BiasMatrix
    BiasTrainingMatrix = np.transpose(BiasTrainingMatrix)
    BiasTestMatrix = np.transpose(BiasTestMatrix)

    # H (Sigmoide function) 
    activationTraining = multiplyMatrixes(trainingX, weights) + BiasTrainingMatrix
    activationTest = multiplyMatrixes(testX, weights) + BiasTestMatrix
    
    H_Training = dotDivide( 1, (1 + np.exp(-activationTraining)) )    
    H_Test = dotDivide( 1, (1 + np.exp(-activationTest)) )

    # Beta
    aux = multiplyMatrixes( np.transpose(H_Training) , H_Training )
    delta = np.identity(aux.shape[0]) * 10e-3

    inverse = np.linalg.inv(dotDivide(np.identity(D) , C) + delta + aux)

    # Complete Formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    Beta = multiplyMatrixes(inverse, np.transpose(H_Training))
    Beta = multiplyMatrixes(Beta, trainingY)

    # Prediction
    predictedLabels = multiplyMatrixes(H_Test, Beta)

    correctPrediction = 0
    for i in range(testY.shape[0]):
      if np.round(predictedLabels[i]) == testY[i]:
        correctPrediction +=1
    
    CCR = correctPrediction / testY.shape[0]
    return CCR * 100

## Genetic Algorithm

In [377]:
def GeneticAlgorithm(population: Population, max_generations:int, optimalD:int, optimalC:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
  t = 0
  print("Steps: \n1. Reproduction \n2. Fitness \n3. Roulette Selection")
  while t < max_generations:
    print("----------------------------------------------------")
    print("Generation: ", t)
      # Reproduction
    ReproducePopulation(population)  
      # Compute Fitness
    ComputePopulationFitness(population, optimalD, optimalC, trainX, trainY, testX, testY)

    #print("Generation: ", t, " Best Fitness: ", population.best_gene.fitness)
      # Selector operator
    
    print(f"Population size before roulette: {population.size}")
    population = RouletteWheelSelection(population)
    print(f"Population size afer roulette: {population.size}")
      # Continue iterating
    t += 1
    print("----------------------------------------------------")
  # end while
  print("Genetic Algorithm done")  
  return population

In [378]:
# GA

size_population = 5
max_generations = 3
crossover_prob = 0.65 # This value and mutation_prob are randomly set at Reproduce Population functino
mutation_prob = 0.15
optimalD = 1000
optimalC = 1

ga_start_execution_time = time.time()
print("Starting GA ...")

# Init population  
print(f"Creating population with {size_population} individuals ... K = {K} - D = {optimalD}")
ga_first_generation = create_population(size_population, K, optimalD)

# Apply Algorithm and Evolve
lastGeneration = GeneticAlgorithm(ga_first_generation, max_generations, optimalD, optimalC, X_trainVal, Y_trainVal, X_testVal, Y_testVal)

# Print lastGeneration best gene
print(lastGeneration.best_gene)

# Train ELM with best W
GA_CCR = TrainModelAndOutputResults(lastGeneration.best_gene, optimalD, optimalC, X_train, Y_train, X_test, Y_test)

print("GA CCR:", GA_CCR, "\nError:", 100-GA_CCR)
print("GA execution time: ", time.time() - ga_start_execution_time, "sec")
print("- - - - - - - -")


Starting GA ...
Creating population with 5 individuals ... K = 21 - D = 1000
Steps: 
1. Reproduction 
2. Fitness 
3. Roulette Selection
----------------------------------------------------
Generation:  0
... Reproducing population ...
0 - [1 0 1 1 1 0 1 0 1 1 0 0 1 0 0 0 0 1 0 0 1]
1 - [0 0 0 1 1 0 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0]
2 - [0 1 0 0 0 1 0 1 1 1 1 0 0 0 1 0 1 1 1 1 0]
3 - [0 1 0 0 1 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 0]
4 - [0 1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 1 0]
Creating two childs!
Creating two childs!
Reproduced final population:
0 - [1 0 1 1 1 0 1 0 1 1 0 0 1 0 0 0 0 1 0 0 1] - fitness: inf
1 - [0 0 0 1 1 0 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0] - fitness: inf
2 - [0 1 0 0 0 1 0 1 1 1 1 0 0 0 1 0 1 1 1 1 0] - fitness: inf
3 - [0 1 0 0 1 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 0] - fitness: inf
4 - [0 1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 1 0] - fitness: inf
5 - [1 0 1 1 1 0 1 0 1 1 0 0 1 1 0 0 0 1 0 0 0] - fitness: inf
6 - [0 1 0 0 0 1 0 1 1 0 1 0 0 0 1 0 1 1 1 1 1] - fitness: inf
7 - [0 0 1 0 

## Particle Swarm Optimization

In [379]:
def PSO_Algorithm(swarm, maxGenerations, w, c1, c2, dampingW, optimalD, optimalC, trainX, trainY, testX, testY):
  t = 0
  while t < maxGenerations:
    # Evaluate Particles' Fitness and update Pb_i and Gb_i
    ComputeSwarmFitness(swarm, optimalD, optimalC, trainX, trainY, testX, testY)
      
    # Update Velocity and Position of each particle
    UpdateVelocitiesAndPositionsPSO(swarm, w, c1, c2)

    # Damping inertia
    w = w * dampingW

    # Increase step
    t += 1

  return 1

In [380]:
# PSO
'''
swarm_size = 2 # total population
num_generations = 1
w = 1 # inertia weight coefficient
dampingW = 0.99 # damping for inertia coefficient
c1 = 2 # cognitive coefficient
c2 = 2 # social coefficient
optimalD = 1000
optimalC = 1

print("Starting PSO method...")
pso_start_execution_time = time.time()
# Init population
swarm = Swarm(swarm_size).initializeSwarm() # Swarm velocity is set to 0 by default
# Apply Algorithm and Iterate
PSO_Algorithm(swarm, num_generations, w, c1, c2, dampingW, optimalD, optimalC, X_trainVal, Y_trainVal, X_testVal, Y_testVal)

print("PSO Completed")
# Train ELM with best W
PSO_CCR = TrainModelAndOutputResults(swarm.global_best_w, optimalD, optimalC, X_train, Y_train, X_test, Y_test)

print("PSO CCR:", PSO_CCR, "\nError:", 100-PSO_CCR)
print("PSO execution time: ", time.time() - pso_start_execution_time, "sec")

# Print best solution found
print("Best feature selection: ", swarm.global_best_c)
print("Best Weights: ", swarm.global_best_w)
'''

'\nswarm_size = 2 # total population\nnum_generations = 1\nw = 1 # inertia weight coefficient\ndampingW = 0.99 # damping for inertia coefficient\nc1 = 2 # cognitive coefficient\nc2 = 2 # social coefficient\noptimalD = 1000\noptimalC = 1\n\nprint("Starting PSO method...")\npso_start_execution_time = time.time()\n# Init population\nswarm = Swarm(swarm_size).initializeSwarm() # Swarm velocity is set to 0 by default\n# Apply Algorithm and Iterate\nPSO_Algorithm(swarm, num_generations, w, c1, c2, dampingW, optimalD, optimalC, X_trainVal, Y_trainVal, X_testVal, Y_testVal)\n\nprint("PSO Completed")\n# Train ELM with best W\nPSO_CCR = TrainModelAndOutputResults(swarm.global_best_w, optimalD, optimalC, X_train, Y_train, X_test, Y_test)\n\nprint("PSO CCR:", PSO_CCR, "\nError:", 100-PSO_CCR)\nprint("PSO execution time: ", time.time() - pso_start_execution_time, "sec")\n\n# Print best solution found\nprint("Best feature selection: ", swarm.global_best_c)\nprint("Best Weights: ", swarm.global_best_

## Coral Reef Optimization

In [381]:
def CRO_Algorithm(reef:Reef, max_generations: int):
  t = 0

  while t < max_generations:
    print("Iteration:", t)
    # Update variables
    F_broadcast = np.random.uniform() # broadcast fraction
    F_brooding = 1 - F_broadcast # brooding fraction
    Pd = 0.2 # predation probability

    # Broadcast corals
    reproduction_candidates = [c for c in reef.gene_list if c is not None]

    [father_candidate, mother_candidate] = np.random.choice(reproduction_candidates, 2, replace=False)
    offsprings = ExternalReproduction(father=father_candidate, mother=mother_candidate)

    print("Father: ", father_candidate.c)
    print("Mother: ", mother_candidate.c)
    
    reproduction_candidates.remove(father_candidate)
    reproduction_candidates.remove(mother_candidate)
    
    #  Brooding
    
    # Asexual reproduction

    # Predation 

    # Fitness Evaluation

    # Increase step
    t += 1
    # - end WHILE

  print("Best CRO: ", "coral X")

In [382]:
def list_splitter(list_to_split, ratio):
    elements = len(list_to_split)
    middle = int(elements * ratio)
    return [list_to_split[:middle], list_to_split[middle:]]

In [383]:
lista = [1,2,3,4,5,6,7,8,9,10]

print(list_splitter(lista, 0.79))
[father,mother] = np.random.choice(lista, 2, replace=True)

print(father)
print(mother)

size_test = 56
randomPercentage = np.random.uniform() 
fraction = int(randomPercentage * size_test)

print("SIZE:", size_test)
print("Percentage:",randomPercentage * 100)
print("Fraction:",fraction)
print("Left:", size_test - fraction)



[[1, 2, 3, 4, 5, 6, 7], [8, 9, 10]]
2
5
SIZE: 56
Percentage: 60.07529216389066
Fraction: 33
Left: 23


In [384]:
# CRO
reef_size = 6
rho0 = 0.8 # free/occupied rate
eta = 2 # larva's attempts
coral_generations = 2 
fraction_broadcast = np.random.uniform() # broadcast fraction
fraction_brooding = 1 - fraction_broadcast # brooding fraction
predation_percentage = 0.2 # predation probability
optimalD = 1000
optimalC = 1

cro_start_execution_time = time.time()
print("- - - - - - - -")
print("Starting CRO method...")
# Init population
reefFirstGeneration = Reef(rho0,reef_size).initializeReef(K,optimalD)

# Print reef
for ind, i in enumerate(reefFirstGeneration.gene_list):
    if i is not None:
        print("Coral: ", ind, " C: ", i.c)
    else:
        print("Coral: ", ind, " C: ", "None")
# Apply Algorithm and Evolve
CRO_Algorithm(reefFirstGeneration, coral_generations)

print("CRO Completed")

print("CRO execution time: ", time.time() - cro_start_execution_time, "sec")
print("- - - - - - - -")

- - - - - - - -
Starting CRO method...


TypeError: __init__() missing 2 required positional arguments: 'K' and 'D'

# MAIN

In [ ]:
if __name__ == '__main__':
  # ! We should set random number generator seed
  #np.random.seed(seed=0)

  # - - ALGORITHMS - - -
  print("Hello guys...")


In [ ]:
gene_list_size = 40

index = 5

random_mother_index = np.random.randint(0,gene_list_size) # Choose a random mother

print("Choosing a mother...")
contador = 0
# Make sure that the mother is a different individual in the population
while(random_mother_index == index): # If the mother is the same as the individual, choose another one

    random_mother_index = np.random.randint(0,gene_list_size)
    contador += 1
    print(f"Contador - {contador}, random_mother_index - {random_mother_index}, index - {index}")

print(f"Mother chosen... {random_mother_index}")